In [102]:
import pandas as pd

import folium

import geopandas as gp

import numpy as np



from scipy.spatial import ConvexHull

import random

from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt

import networkx as nx

import math

# from osgeo import ogr

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter


import h3


In [103]:
'''Eliminate POLYGON'''

def Get_POLYGON(coords):
    if coords.type=='Polygon':
        return coords
    else:
        Score={i:coords[i].area for i in range(len(coords))}
        idx=max(Score, key=Score.get)
        return coords[idx]
    
'''Get Neighbor ranges'''

def Get_neighbors(y):
    x=list()
    for y_ in y:
        for y__ in y_:
            x.append(y__)
    return x
    

In [111]:
'''1. Dividing NYC to Zones'''

Zone_data = gp.read_file('./Data/NYC_Zones/geo_export_789b911c-2a6d-4092-a682-6c5e6ce53409.shp')

Zone_data=Zone_data[['borough','zone','geometry']]

Zone_data=Zone_data.loc[Zone_data['borough']=='Manhattan']

Zone_data=Zone_data.reset_index(drop=True)

Zone_data['Zone_id']=['Zone_'+str(i) for i in Zone_data.index]

Zone_data=Zone_data[['Zone_id','borough','zone','geometry']]

# Islands=['Zone_19','Zone_20','Zone_21','Zone_38','Zone_47','Zone_48']

Islands=['Zone_19','Zone_20','Zone_21']

Zone_data=Zone_data.loc[~Zone_data['Zone_id'].isin(Islands)]

Zone_data=Zone_data.reset_index(drop=True)

Zone_data['Zone_id']=['Zone_'+str(i) for i in Zone_data.index]

Zone_data['geometry']=Zone_data.apply(lambda x:Get_POLYGON(x['geometry']),axis=1)

Zone_data['center'] = Zone_data.apply(lambda x:x['geometry'].centroid,axis=1)

Zone_data

,Zone_id,borough,zone,geometry,center
0,Zone_0,Manhattan,Alphabet City,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",POINT (-73.97696825691759 40.72375214158459)
1,Zone_1,Manhattan,Bloomingdale,"POLYGON ((-73.95954 40.79872, -73.96004 40.798...",POINT (-73.96547951516966 40.80197049389943)
2,Zone_2,Manhattan,Battery Park,"POLYGON ((-74.01566 40.70483, -74.01540 40.704...",POINT (-74.01556351255894 40.70294583498452)
3,Zone_3,Manhattan,Battery Park City,"POLYGON ((-74.01244 40.71906, -74.01282 40.717...",POINT (-74.01607915192938 40.71203815707432)
4,Zone_4,Manhattan,Central Harlem,"POLYGON ((-73.94774 40.80960, -73.94506 40.808...",POINT (-73.95129200243512 40.80433394979442)
5,Zone_5,Manhattan,Chinatown,"POLYGON ((-73.99750 40.71407, -73.99744 40.714...",POINT (-73.99815149899052 40.71245931177932)
6,Zone_6,Manhattan,Central Harlem North,"POLYGON ((-73.93436 40.83620, -73.93445 40.835...",POINT (-73.94077166756259 40.81825786111904)
7,Zone_7,Manhattan,Central Park,"POLYGON ((-73.97255 40.76490, -73.97301 40.764...",POINT (-73.96555356545908 40.78247809974769)
8,Zone_8,Manhattan,Clinton East,"POLYGON ((-73.99118 40.75533, -73.99163 40.754...",POINT (-73.989844643133 40.76225275531919)
9,Zone_9,Manhattan,Clinton West,"POLYGON ((-73.99668 40.77351, -73.99434 40.772...",POINT (-73.9951352207551 40.76623772504166)


In [112]:
'''Background'''

background = folium.Map([40.769602, -73.973667],tiles='CartoDB positron',zoom_start=12)

'''Zones'''

folium.Choropleth(
    geo_data=Zone_data[['geometry']],
    fill_color='blue',
    fill_opacity=0.5,
    name='Zone').add_to(background)


folium.LayerControl().add_to(background)


background

In [117]:
'''2. Dividing NYC into Hexagons'''

resolution = 9

Grids=list()

for idx,row in Zone_data.iterrows():
    
    boundary=[[[x[1],x[0]] for x in list(row['geometry'].exterior.coords)]]
    
    geoJson = {'type': 'Polygon','coordinates': boundary}
    
    Grids += list(h3.polyfill(geoJson, resolution))
    
Grids=list(set(Grids))

Complementary=list()

for grid in Grids:
    
    Complementary+=Get_neighbors(h3.hex_range_distances(grid, 1))
    
    
'''Revised'''
    
lat,lng=40.768348, -74.000557

h3_grid=h3.geo_to_h3(lat, lng, resolution)

Complementary.append(h3_grid)

lat,lng=40.764503, -73.946362

h3_grid=h3.geo_to_h3(lat, lng, resolution)

Complementary.append(h3_grid)

lat,lng=40.762268, -73.948302

h3_grid=h3.geo_to_h3(lat, lng, resolution)

Complementary.append(h3_grid)

Complementary=list(set(Complementary))


'''Visualizaton'''

Grids_data=pd.DataFrame(Complementary,columns=['grid'])

Grids_data['geometry']=Grids_data.apply(lambda x:[[float(x[1]),float(x[0])] for x in h3.h3_to_geo_boundary(x['grid'])],axis=1)

Grids_data['center']=Grids_data.apply(lambda x:h3.h3_to_geo(x['grid']),axis=1)

GeoData=list()
         
for idx,row in Grids_data.iterrows():
    
    GeoData.append({"type":"Feature",\
                    "id":row['grid'],\
                    "geometry":{"type":"Polygon",\
                                "coordinates":[row['geometry']]
                               }
                   })
    
GeoJson={"type":"FeatureCollection","features":GeoData}

GeoJson

'''Background'''

background = folium.Map([40.769602, -73.973667],tiles='CartoDB positron',zoom_start=13)

'''Zones'''

choropleth1 =folium.Choropleth(
    geo_data=GeoJson,
    fill_color='blue',
    fill_opacity=0.2,
    name='Zone').add_to(background)

folium.LayerControl().add_to(background)


np.save('./Data/NYC_Network/Grids',Complementary)

background


In [120]:
'''3. Constructing Road Network in Hexagons'''

Street_data = gp.read_file('./Data/NYC_Street/geo_export_2d0a9cb9-782e-42c0-ba15-7345a0f64002.shp')

Street_data=Street_data.loc[(Street_data['borocode']=='1')&(Street_data['rw_type'].isin([1,3]))&(Street_data['status']=='2')]

Street_data=Street_data[['st_name','geometry','shape_leng']]

Street_data=Street_data.rename(columns={'shape_leng':'length'})

Street_data['start']=Street_data.apply(lambda x:list(x['geometry'].coords)[0][::-1],axis=1)

Street_data['end']=Street_data.apply(lambda x:list(x['geometry'].coords)[-1][::-1],axis=1)

Street_data['start_grid']=Street_data.apply(lambda x:h3.geo_to_h3(x['start'][0],x['start'][1],resolution),axis=1)

Street_data['end_grid']=Street_data.apply(lambda x:h3.geo_to_h3(x['end'][0],x['end'][1],resolution),axis=1)

Street_data=Street_data.loc[(Street_data['start_grid'].isin(Complementary))&(Street_data['end_grid'].isin(Complementary))]         
                        
Street_data=Street_data.reset_index(drop=True)

Street_data['Link_id']=['Link_'+str(i) for i in Street_data.index]


'''Encode'''

Get_Str=lambda x,pos:str(x[pos][0])+'&'+str(x[pos][1])

Street_data['start_str']=Street_data.apply(lambda x:Get_Str(x,'start'),axis=1)

Street_data['end_str']=Street_data.apply(lambda x:Get_Str(x,'end'),axis=1)

'''Point list'''

Ponits_str=list(set(list(Street_data['start_str'])+list(Street_data['end_str'])))

Ponits_dic={Ponits_str[i]:'Point_'+str(i) for i in range(len(Ponits_str))}

Points_list=list(Ponits_dic.values())

'''Look up'''

Street_data['start_point']=Street_data.apply(lambda x:Ponits_dic[x['start_str']],axis=1)

Street_data['end_point']=Street_data.apply(lambda x:Ponits_dic[x['end_str']],axis=1)

Street_data




,st_name,geometry,length,start,end,start_grid,end_grid,Link_id,start_str,end_str,start_point,end_point
0,BENNETT,"LINESTRING (-73.93666 40.85045, -73.93620 40.8...",735.721079,"(40.85044732520501, -73.93666142761384)","(40.852335152313174, -73.93571740885848)",892a100ae77ffff,892a100ae0fffff,Link_0,40.85044732520501&-73.93666142761384,40.852335152313174&-73.93571740885848,Point_4589,Point_343
1,STONE,"LINESTRING (-74.01007 40.70455, -74.00975 40.7...",122.538700,"(40.704545914131955, -74.01006968341342)","(40.70477524791872, -74.00974639311217)",892a1072817ffff,892a1072817ffff,Link_1,40.704545914131955&-74.01006968341342,40.70477524791872&-74.00974639311217,Point_3979,Point_1787
2,GOLD,"LINESTRING (-74.00684 40.70786, -74.00653 40.7...",159.149459,"(40.707857835737876, -74.0068377870997)","(40.70822712625459, -74.00653116871753)",892a1072887ffff,892a1072887ffff,Link_2,40.707857835737876&-74.0068377870997,40.70822712625459&-74.00653116871753,Point_3572,Point_4582
3,109,"LINESTRING (-73.94969 40.79622, -73.94919 40.7...",508.092565,"(40.79621698487826, -73.94968682996546)","(40.795536417729416, -73.94808516529754)",892a1008d73ffff,892a1008d0fffff,Link_3,40.79621698487826&-73.94968682996546,40.795536417729416&-73.94808516529754,Point_6328,Point_1899
4,AMSTERDAM,"LINESTRING (-73.93192 40.84671, -73.93185 40.8...",38.069869,"(40.8467105236121, -73.93191507377053)","(40.84680263276957, -73.93185010295767)",892a100ae2bffff,892a100ae2bffff,Link_4,40.8467105236121&-73.93191507377053,40.84680263276957&-73.93185010295767,Point_1966,Point_4211
5,EIGHTH,"LINESTRING (-73.99575 40.74905, -73.99530 40.7...",257.894368,"(40.749052196220944, -73.99574855573138)","(40.74967168209748, -73.99529821527037)",892a100d2cbffff,892a100d2cbffff,Link_5,40.749052196220944&-73.99574855573138,40.74967168209748&-73.99529821527037,Point_721,Point_193
6,61,"LINESTRING (-73.96107 40.76086, -73.95881 40.7...",714.075356,"(40.76085809262594, -73.96106754453669)","(40.75991278601397, -73.95880955474563)",892a100d6b3ffff,892a100d6b7ffff,Link_6,40.76085809262594&-73.96106754453669,40.75991278601397&-73.95880955474563,Point_3129,Point_3358
7,GREENWICH,"LINESTRING (-74.00680 40.73572, -74.00662 40.7...",262.936962,"(40.73572292371414, -74.00679907002781)","(40.736431696922914, -74.006620319663)",892a10721afffff,892a10721afffff,Link_7,40.73572292371414&-74.00679907002781,40.736431696922914&-74.006620319663,Point_6264,Point_2741
8,3,"LINESTRING (-73.96111 40.76906, -73.96060 40.7...",286.455576,"(40.76906440780265, -73.96110997058013)","(40.76974918670643, -73.9606018063753)",892a100d693ffff,892a100892fffff,Link_8,40.76906440780265&-73.96110997058013,40.76974918670643&-73.9606018063753,Point_6305,Point_494
9,HARLEM RIV,"LINESTRING (-73.93483 40.81406, -73.93469 40.8...",74.751585,"(40.814055471812985, -73.93483229160476)","(40.814230630076366, -73.93469166469954)",892a100ab73ffff,892a100ab73ffff,Link_9,40.814055471812985&-73.93483229160476,40.814230630076366&-73.93469166469954,Point_365,Point_5363


In [133]:
'''Save the processed data'''

Link_Point={}

Link_geometry={}

Point_coordinate={}

Point_link={pnt:list() for pnt in Points_list}

Grid_Point={grid:list() for grid in Complementary}

for idx,row in Street_data.iterrows():
    
    Link_Point[row['Link_id']]=(row['start_point'],row['end_point'],row['length'])
    
    Link_geometry[row['Link_id']]=row['geometry']
    
    Point_coordinate[row['start_point']]=row['start']
    
    Point_coordinate[row['end_point']]=row['end']
    
    Point_link[row['start_point']].append(row['Link_id'])
    
    Point_link[row['end_point']].append(row['Link_id'])
    

Point_Grid={k:h3.geo_to_h3(v[0],v[1],resolution) for k,v in Point_coordinate.items()} 

for p,g in Point_Grid.items():
    
    Grid_Point[g].append(p)    
        
np.save('./Data/NYC_Network/Points_list',Points_list)

np.save('./Data/NYC_Network/Point_coordinate',Point_coordinate)

np.save('./Data/NYC_Network/Point_Grid',Point_Grid)
 
np.save('./Data/NYC_Network/Grid_Point',Grid_Point)

Point_link



{'Point_0': ['Link_2192', 'Link_6317', 'Link_7163'],
 'Point_1': ['Link_1733', 'Link_3912', 'Link_5815', 'Link_9056'],
 'Point_2': ['Link_594', 'Link_6021'],
 'Point_3': ['Link_2255', 'Link_5524', 'Link_5545'],
 'Point_4': ['Link_2928', 'Link_6059', 'Link_7138'],
 'Point_5': ['Link_152', 'Link_2444', 'Link_5166', 'Link_7782'],
 'Point_6': ['Link_846', 'Link_1196', 'Link_4788'],
 'Point_7': ['Link_1157', 'Link_3061', 'Link_7451', 'Link_8134'],
 'Point_8': ['Link_5343'],
 'Point_9': ['Link_1219', 'Link_2921', 'Link_9062'],
 'Point_10': ['Link_1538', 'Link_3634', 'Link_9493', 'Link_9915'],
 'Point_11': ['Link_996', 'Link_1923', 'Link_3038', 'Link_10004'],
 'Point_12': ['Link_211', 'Link_3057', 'Link_3331'],
 'Point_13': ['Link_3462', 'Link_4115'],
 'Point_14': ['Link_413', 'Link_4006'],
 'Point_15': ['Link_780', 'Link_1859', 'Link_5660', 'Link_5718'],
 'Point_16': ['Link_274', 'Link_8033', 'Link_9822'],
 'Point_17': ['Link_1439', 'Link_1571', 'Link_2919', 'Link_7004', 'Link_8691'],
 'Poin

In [136]:
'''Background'''

background = folium.Map([40.769602, -73.973667],tiles='CartoDB positron',zoom_start=12)

'''Zones'''

folium.Choropleth(
    geo_data=GeoJson,
    fill_color='blue',
    fill_opacity=0.05,
    name='Grids').add_to(background)

'''Points demo'''

for pnt in Grid_Point['892a100ae77ffff']:
    
    for link in Point_link[pnt]:
        
        folium.PolyLine([x[::-1] for x in list(Link_geometry[link].coords)],\
                    color="green",\
                    weight=2.0,\
                    opacity=1.0).add_to(background)
        
    folium.Circle(
    radius=5,
    location=Point_coordinate[pnt],
    color='red',
    fill=True,
    ).add_to(background)

folium.LayerControl().add_to(background)

background